In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score, accuracy_score, precision_score

sns.set()
pd.set_option("display.max_columns", None)

In [ ]:
target = 'cliked'

In [ ]:
# with open('data/context_history.json', 'r') as file:
#     parsed_json = json.load(file)
# df_context = pd.json_normalize(parsed_json)

In [ ]:
# train, test = train_test_split(df_context, test_size = 0.3, random_state = 456)
# test['schema'] = None
# test['top_1_product'] = None
# test['top_2_product'] = None
# test['top_3_product'] = None
# test['top_4_product'] = None
# test['top_5_product'] = None
# test['is_banner'] = None
# test['products_offered'] = None
# test['cta_number'] = None
# test['id'] = None

In [ ]:
# train.to_pickle("data/train.pkl")
# test.to_pickle("data/test.pkl")

In [ ]:
train = pd.read_pickle("data/train.pkl")
test = pd.read_pickle("data/test.pkl")

In [ ]:
import json
import pandas as pd
with open('data/offer_features.json', 'r') as file:
    parsed_json = json.load(file)
df_offer = pd.json_normalize(parsed_json, 'features', ['id'])

In [ ]:
train = pd.merge(train, df_offer, left_on='offer_id', right_on='id', how='left')

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer

from src.pipeline.utils import DataFrameTransformer, ColumnDropper
from src.pipeline.preprocessing import MapOfferValues, BannerEncoder, ColTypeFromater, DataTranform, ValEncoder, TrigonometricTransformer

Outliers na zbiorze treningowym

In [ ]:
train = train[train['context.booking.param5'] <= 2]
train = train[train['context.booking.param6'] <= 8]
train = train[train['context.booking.param8'] <= 8]

Map offer info

In [ ]:
map_values = MapOfferValues()

In [ ]:
mapping_pipeline = Pipeline([
    ("map_offer_values", map_values),
])

Imputation

In [ ]:
# impute_mode = ['context.booking.param8']
# impute_binary = ['is_banner']
# impute_median = ['context.leg.param3']
# mode_imputer = DataFrameTransformer(SimpleImputer(strategy='most_frequent'), impute_mode)
# zero_imputer = DataFrameTransformer(SimpleImputer(strategy='constant', fill_value=0), impute_binary)
# median_imputer = DataFrameTransformer(SimpleImputer(strategy='median'), impute_median)

In [ ]:
# imputation_pipeline = Pipeline([
#     ("mode_imputer", mode_imputer),
#     ("zero_imputer", zero_imputer),
#     ("median_imputer", median_imputer),
# ])

# nie ma braków danych

Preprocessing

In [ ]:
is_banner_encoder = BannerEncoder()
date_encoder = DataTranform()
date_encoder_tryg = TrigonometricTransformer()
value_encoder = ValEncoder()
type_formater = ColTypeFromater()

In [ ]:
preprocessing_pipeline = Pipeline([
    ("is_banner_encoder", is_banner_encoder),
    ("date_encoder", date_encoder),
    ("date_encoder_tryg",date_encoder_tryg),
    ("value_encoder", value_encoder),
    ("type_formater", type_formater),
])

New Features

In [ ]:
from src.pipeline.preprocessing import ThreeMonthsAggregator, SixMonthsAggregator

In [ ]:
aggregator_3m = ThreeMonthsAggregator()
aggregator_6m = SixMonthsAggregator()

In [ ]:
new_features_pipeline = Pipeline([
    ("aggregator_3m", aggregator_3m),
    ("aggregator_6m", aggregator_6m),
])

Encodig

In [ ]:
from category_encoders import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

def convert_to_str(x):
    return x.astype(str)

In [ ]:
encode_one_hot =['schema', 'top_1_product', 'top_2_product', 'top_3_product', 'top_4_product', 'top_5_product', 
                 'context.booking.param5', 'context.booking.param3', 'context.booking.param9',
                 'context.booking.param10', 'context.booking.param12',
                 'context.booking.param13', 'context.booking.param15',
                 'context.leg.param8', 'context.leg.param10', 'offer_id'
                
                ]

category_creator = DataFrameTransformer(FunctionTransformer(func=convert_to_str), encode_one_hot)
one_hot_encoder = OneHotEncoder(cols=encode_one_hot)
encoding_pipeline = Pipeline([
      ('one_hot_encoder', one_hot_encoder),
])

Pipeline

In [ ]:
to_drop = ['id','context.param2', "request_dttm", 'context.booking.param11', 'context.booking.param14', 'context.booking.param16', 'context.leg.param12', 'context.leg.param13', 'context.leg.param14', 'context.leg.param15', 'context.param3' ]
# zmienna z jednym poziomem, tekstowe bez kategoryzacji, data

In [ ]:
pipeline = Pipeline([
    ("mapping_step", mapping_pipeline),
    ("preprocessing_step", preprocessing_pipeline),
    ("encoding_step", encoding_pipeline),
    ("feature_engineering_step", new_features_pipeline),
    ("drop_columns", ColumnDropper(input_cols=to_drop)),
])

In [ ]:
abt_train = pipeline.fit_transform(train)

In [ ]:
abt_train.columns.tolist()

In [ ]:
abt_test = pipeline.transform(test)

In [ ]:
predictors = abt_test.columns.tolist()
to_drop = ["cliked", 'id', "offer_id"]
for element in to_drop:
    while element in predictors:
            predictors.remove(element)

In [ ]:
X_train = abt_train[predictors]
X_test = abt_test[predictors]

In [ ]:
y_train = abt_train[['cliked']]
y_test = abt_test[['cliked']]

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, space_eval
from hyperopt.pyll import scope

space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 50, 200, q=50)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.2)),
    'max_depth': scope.int(hp.quniform('max_depth', 3, 10, q=1)),
    'min_child_weight': hp.loguniform('min_child_weight', np.log(1e-5), np.log(0.2)),
    'subsample': hp.uniform('subsample', 0.6, 0.8),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'reg_alpha': hp.loguniform('reg_alpha', np.log(1e-9), np.log(1)),
    'reg_lambda': hp.loguniform('reg_lambda', np.log(1e-9), np.log(1)),
    'gamma': hp.loguniform('gamma', np.log(0.1), np.log(10)),
    'booster': hp.choice('booster', ['gbtree', 'dart'])
}

optymalizacja hiperparametrow

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import f1_score, recall_score, precision_score, log_loss

# def get_metrics(model, X, y):
#     predictions = model.predict(X)
#     f1 = f1_score(y, predictions)
#     recall = recall_score(y, predictions)
#     logloss = log_loss(y, model.predict_proba(X))
    
#     return {'f1': f1, 'recall': recall, 'logLoss': logloss, 'predictions': predictions}

# metrics = []

# def train_model(params, n_split = 5):
#     scores = list()
    
#     cv= StratifiedKFold(n_splits= n_split, shuffle = True)
    
#     for train_idx, val_idx in cv.split(X_train, y_train):
#         tr, y_tr = X_train.iloc[train_idx], y_train.iloc[train_idx][target]
#         val, y_val = X_train.iloc[val_idx], y_train.iloc[val_idx][target]
    
#     model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', tree_method='approx')
#     model.set_params(**params)
#     model.fit(tr, y_tr, verbose=False)
    
#     metrics.append(get_metrics(model, tr, y_tr))
#     scores.append(get_metrics(model, val, y_val)['logLoss'])

#     return np.average(scores)

# trials = Trials()

# best_params = fmin(
#         fn=train_model,
#         space=space,
#         algo=tpe.suggest,
#         max_evals=10,
#         trials=trials
#     )

In [ ]:
#best_params

In [ ]:
params = dict(
 colsample_bytree = 0.6786466169745081,
 gamma = 1.2366943011928748,
 learning_rate = 0.00805577148504449,
 max_depth = 6,
 min_child_weight = 0.0066123836862955945,
 n_estimators= 100,
 reg_alpha = 0.3118157899554353,
 reg_lambda = 0.15780057333621578,
 subsample = 0.6985172214982275)

In [ ]:
model = XGBClassifier()
model.set_params(**params)
model.fit(X_train, y_train, verbose=False)

In [ ]:
predictions_test = model.predict(X_test)
recall = recall_score(y_test, predictions_test)
f1 = f1_score(y_test, predictions_test)
accuracy = accuracy_score(y_test, predictions_test)
precision = precision_score(y_test, predictions_test)

print(f"recall: {recall}, f1: {f1}, accuracy: {accuracy}, precision: {precision}")

In [ ]:
def get_probability(df, offert_id, model, pipeline):
    _df = df.copy()
    _df['id'] = offert_id
    _df[['top_1_product', 'top_2_product', 'top_3_product', 'top_4_product', 'top_5_product']] = _df['id'].str.split('_').apply(lambda x: pd.Series(x[2:7]))
    _df[['schema']] = _df['id'].str.split('_').apply(lambda x: pd.Series(x[1]))
    X = pipeline.transform(_df.drop(columns=['id']))
    prob = model.predict_proba(X)[:,1][0]
    return prob

Results

In [ ]:
test_one = test[:1]

In [ ]:
lista_ofert = ['OFFER_A_1_2_3_4_5','OFFER_A_2_1_3_4_5','OFFER_A_3_1_4_2_5','OFFER_A_4_1_2_3_5','OFFER_A_5_1_2_3_4','OFFER_A_5_2_4_1_3','OFFER_B_1_2_3_4_5','OFFER_B_2_1_3_4_5','OFFER_B_3_1_4_2_5','OFFER_B_4_1_2_3_5','OFFER_B_5_1_2_3_4','OFFER_B_5_2_4_1_3','OFFER_C_1_2_3_4_5','OFFER_C_2_1_3_4_5','OFFER_C_3_1_4_2_5','OFFER_C_4_1_2_3_5','OFFER_C_5_1_2_3_4','OFFER_C_5_2_4_1_3','OFFER_D_1_2_3_4_5','OFFER_D_2_1_3_4_5','OFFER_D_3_1_4_2_5','OFFER_D_4_1_2_3_5','OFFER_D_5_2_4_1_3','OFFER_E_1_2_3_4_5','OFFER_E_2_1_3_4_5','OFFER_E_3_1_4_2_5','OFFER_E_4_1_2_3_5','OFFER_E_5_1_2_3_4','OFFER_E_5_2_4_1_3']

In [ ]:
def get_probabilities_for_offert_ids(df, offert_ids, model, pipeline, predictors):
    result_list = []
    for offert_id in offert_ids:
        _df = df.copy()
        _df['id'] = offert_id
        _df[['top_1_product', 'top_2_product', 'top_3_product', 'top_4_product', 'top_5_product']] = _df['id'].str.split('_').apply(lambda x: pd.Series(x[2:7]))
        _df[['schema']] = _df['id'].str.split('_').apply(lambda x: pd.Series(x[1]))
        X = pipeline.transform(_df.drop(columns=['id']))
        X = X[predictors]
        prob = model.predict_proba(X)[:, 1][0]
        result_dict = {'id': offert_id, 'probability': np.float64(prob)}
        result_list.append(result_dict)
    sorted_results = sorted(result_list, key=lambda x: x['probability'], reverse=True)[:3]
    result_dict = {"ranking": sorted_results}
    return result_dict

Print best offers

In [ ]:
get_probabilities_for_offert_ids(test_one, lista_ofert, model, pipeline, predictors)

In [ ]:
json.dumps(get_probabilities_for_offert_ids(test_one, lista_ofert, model, pipeline, predictors), indent=2)